## Exploratory Data Analysis
Each row represents a customer, each column contains attributes related to customer demographics and previous transactions with the bank.

A Bank wants to take care of customer retention for their product; savings accounts. The bank wants you to identify customers likely to churn balances below the minimum balance. You have the customers information such as age, gender, demographics along with their transactions with the bank. Your task as a data scientist would be to predict the propensity to churn for each customer.


## Data Dictionary

There are multiple variables in the dataset which can be cleanly divided in 3 categories:

### Demographic information about customers

<b>customer_id</b> - Customer id

<b>vintage</b> - Vintage of the customer with the bank in number of days

<b>age</b> - Age of customer

<b>gender</b> - Gender of customer

<b>dependents</b> - Number of dependents

<b>occupation</b> - Occupation of the customer 

<b>city</b> - City of customer (anonymised)


### Bank Related Information for customers


<b>customer_nw_category</b> - Net worth of customer (3:Low 2:Medium 1:High)

<b>branch_code</b> - Branch Code for customer account

<b>days_since_last_transaction</b> - No of Days Since Last Credit in Last 1 year


### Transactional Information

<b>current_balance</b> - Balance as of today

<b>previous_month_end_balance</b> - End of Month Balance of previous month


<b>average_monthly_balance_prevQ</b> - Average monthly balances (AMB) in Previous Quarter

<b>average_monthly_balance_prevQ2</b> - Average monthly balances (AMB) in previous to previous quarter

<b>percent_change_credits</b> - Percent Change in Credits between last 2 quarters

<b>current_month_credit</b> - Total Credit Amount current month

<b>previous_month_credit</b> - Total Credit Amount previous month

<b>current_month_debit</b> - Total Debit Amount current month

<b>previous_month_debit</b> - Total Debit Amount previous month

<b>current_month_balance</b> - Average Balance of current month

<b>previous_month_balance</b> - Average Balance of previous month

<b>churn</b> - Average balance of customer falls below minimum balance in the next quarter (1/0)


Wow, we can already see there are many features in the data dictionary which we included in our hypothesis.

Now, we will begin the exploratory analysis of the dataset. Note that here I have used a popular visualization library called <b>Seaborn</b> which we covered in a previous module. Seaborn is library that is easy to use and has functionality to plot a variety of plots with few lines of code. 
* Here is the link to documentation for Seaborn: https://seaborn.pydata.org/index.html. 
* You can also go through this article for a quick overview on visualization and Seaborn: https://www.analyticsvidhya.com/blog/2015/04/comprehensive-guide-data-exploration-sas-using-python-numpy-scipy-matplotlib-pandas/

### Loading Packages
Let us load the packages needed for visualization and exploratory analysis

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
sns.set(style="white")


### Loading Data
Loading the csv as a dataframe and checking structure of the dataset

In [2]:
df = pd.read_csv("churn_prediction.csv")

FileNotFoundError: [Errno 2] File churn_prediction.csv does not exist: 'churn_prediction.csv'

In [ ]:
df.shape, df.columns

We have data for 28382 customers with 22 columns. So, essentially we have 21 features and 1 target column which is churn. Let us quickly look at the values for each column

In [ ]:
df.iloc[1,:]

Alright. Here, we have a mix of categorical, numerical and ordinal variables as shown. There are missing values also in some of the features and we would treat them as a part of preprocessing step when we build the model.

<h3> Target Exploration</h3>

We are trying to predict if the client's account balance drops below the minimum balance prescribed for the customer. 

Clearly, this is a binary classification problem. Let's look at the target variable and find out how many customers are in the churn category

In [ ]:
ax = sns.catplot(y="churn", kind="count", data=df, height=2.6, aspect=2.5, orient='h')

In [ ]:
df['churn'].value_counts(normalize = True)

## Numerical features

Let us look at the numerical features. From the description provided in the data dictionary and cell 8, we can see that we have the following numerical features. Let us quickly describe them to check the following:

* Count: Can be used to check for missing value count
* Mean: Mean of the variable
* Standard Deviation: Standard deviation of the variable
* Minimum: Minimum value of the variable
* Quantile values: 25, 50 (median) & 75% quantiles of the variable
* Maximum: Maximum value of the variable

Notice that we will not directly used dtypes function to identify numerical columns but rather used business sense to select numerical features as we have seen from a smaple record, branch code and city code actually represent categories and not some meaningful numerical value.

In [ ]:
df.dtypes

As shown dtypes function puts city and branch code features in the numerical category but that is not the intention

In [ ]:
numerical_cols = ['customer_id', 'vintage', 'age', 'dependents', 
       'customer_nw_category', 'days_since_last_transaction', 'current_balance',
       'previous_month_end_balance', 'average_monthly_balance_prevQ', 'average_monthly_balance_prevQ2',
       'current_month_credit',
       'previous_month_credit', 'current_month_debit', 'previous_month_debit',
       'current_month_balance', 'previous_month_balance']
df[numerical_cols].describe()

Lets list down a few key observations:

* Customer ID here is just an id variable identifying a unique customer and has values between 1 and 30301
* On an average, a customer from this set has been with the bank for 2400 days or around 6.5 years
* On an average, a customer has less than 1 dependent and has an average age of 48 years
* A general trend on variables which are related to balances have a wide range with huge outliers, it will key to observe these outliers
* Most of the customers lie in category 2 or 3 for net worth and have on an average done the last transaction 70 days ago. Now the high net worth customers (Category) must have high credit, debit and balance values. Let's verify this using data.

### Customer Net worth Category &  Balance Features

We will use a groupby function to check the mean values of balance features.

In [ ]:
cols = ['current_balance',
       'previous_month_end_balance', 'average_monthly_balance_prevQ', 'average_monthly_balance_prevQ2',
        'current_month_credit','previous_month_credit', 'current_month_debit', 'previous_month_debit',
       'current_month_balance', 'previous_month_balance']
df.groupby(['customer_nw_category'])[cols].mean()

So there is clear consistency here as mean values of balance features and the credit/debit features have higher values for net worth category 1 and lower value for net worth categories 2 & 3.

The bulk of features are comprised of balance and credit debit features. Let us explore them in detail

### Balance & Credit/Debit Features

We will start by looking at average balance in the current month. We will use a histogram to check its distribution.

#### Average Monthly Balance Features

In [ ]:
sns.distplot(df['current_month_balance'], kde = False)
plt.show()

Due to the huge outliers in both positive and negative directions, it is very difficult to derive insights from this plot. 

* In this case, we could convert such columns to log and then check the distributions. 
* However, since there are negative values, it cannot be a direct log conversion as log of negative numbers is not defined.
* To tackle this, we add a positive constant within the log as a correction

In [ ]:
# To account for negative values we add a constant value within log
temp = np.log(df['current_month_balance'] + 6000) 

sns.distplot(temp, kde = False, bins = 100)
plt.show()

Now, we can see more clearly that this is a right skewed feature and we have much more clarity on its distribution. Let us use subplot to quickly look at more numerical features together and see trends.

In [ ]:
# Numerical Features
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 6))
xmin = 7
xmax = 16
# Current Month Average Balance
temp = np.log(df['current_month_balance'] + 6000) # To account for negative values we add a constant value within log
ax1.set_xlim([xmin,xmax])
ax1.set(xlabel='log of average balance of current month')
sns.distplot(temp, kde = False, bins = 200, ax = ax1)


# Previous month average balance
temp = np.log(df['previous_month_balance'] + 6000) # To account for negative values we add a constant value within log
ax2.set_xlim([xmin,xmax])
ax2.set(xlabel='log of average balance of previous month')
sns.distplot(temp, kde = False, bins = 200, ax = ax2)


plt.show()

#### Current Balance today vs Average Monthly Balance in current month

As expected the average monthly balance for both months are quite similar and have right skewed histograms as shown. Now let us compare the current month average balance vs current balance as of today

In [ ]:
# Numerical Features
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 6))
xmin = 7
xmax = 16
# Current Month Average Balance
temp = np.log(df['current_month_balance'] + 6000) # To account for negative values we add a constant value within log
ax1.set_xlim([xmin,xmax])
#ax1.set(xlabel='log of average balance of current month')
sns.distplot(temp, kde = False, bins = 200, ax = ax1)


# Current End of month average balance
temp = np.log(df['current_balance'] + 6000) # To account for negative values we add a constant value within log
ax2.set_xlim([xmin,xmax])
#ax2.set(xlabel='log of month end balance of current  month')
sns.distplot(temp, kde = False, bins = 200, ax = ax2)


plt.show()

* Here, we can see that the distribution for both lie in almost the same interval, however, there are larger number of values for current balance just below 9 which might have been contributed by the churning customers. 

* It might be a good idea to create a feature which is the difference of these 2 variables during the model building process.

* Students are encouraged to do more univariate analysis on other balances and check distributions to find similar insights. 

Next, in order to understand which of these features might be important to predict the churn, we will do a bivariate analysis.

### Bivariate Analysis

Now, we will check the relationship of the numeric variables along with the target. Again conversion to log is important here as we have a lot of outliers and visualization will be difficult for it.

#### Churn vs Current & Previous month balances

In [ ]:
balance_cols = ['current_balance','previous_month_end_balance',
                'current_month_balance', 'previous_month_balance']
df1 = pd.DataFrame()

for i in balance_cols:
    df1[str('log_')+ i] = np.log(df[i] + 6000)

log_balance_cols = df1.columns

In [ ]:
df1['churn'] = df['churn']

We will use the brilliant pairplot function from Seaborn which supports displaying relationship between multiple variables. It displays the scatter plot between a pair of feature and also displays the distribution

Here I have included the following:
* Log of current balance & previous month end balance
* Log of average monthly balance of current and previous month
* Churn is represented by the color here (Orange - Churn, Blue - Not Churn)

In [ ]:
#sns.pairplot(df1,vars=log_balance_cols, hue = 'churn',plot_kws={'alpha':0.1})
df1_no_churn = df1[df1['churn'] == 0]
sns.pairplot(df1_no_churn,vars=log_balance_cols,plot_kws={'alpha':0.1})
plt.show()

In [ ]:
#sns.pairplot(df1,vars=log_balance_cols, hue = 'churn',plot_kws={'alpha':0.1})
df1_churn = df1[df1['churn'] == 1]
sns.pairplot(df1_churn,vars=log_balance_cols,plot_kws={'alpha':0.1})
plt.show()

In [ ]:
sns.pairplot(df1,vars=log_balance_cols,hue ='churn',plot_kws={'alpha':0.1})
plt.show()

The distribution for these features look similar. We can make the following conclusions from this:
* There is high correlation between the previous and current month balances which is expected
* The lower balances tend to have higher number of churns which is clear from the scatter plots
* Distribution for the balances are all right skewed

### Credit and Debits for current and previous months

Total credit and debit amounts for the current and previous can be clubbed into the same category. Let us again use the pair plot to check distributions and scatter plots.

In [ ]:
cr_dr_cols = ['current_month_credit','previous_month_credit', 
              'current_month_debit', 'previous_month_debit']
df1 = pd.DataFrame()

for i in cr_dr_cols:
    df1[str('log_')+ i] = np.log(df[i])

log_dr_cr_cols = df1.columns

In [ ]:
df1['churn'] = df['churn']

In [ ]:
sns.pairplot(df1,vars=log_dr_cr_cols, hue = 'churn',plot_kws={'alpha':0.5})
plt.show()

Both credit and debit patterns show significant difference in distributions for churned and non churned customers.
* Bimodal distribution/Double Bell Curve shows that there are 2 different types of customers with 2 brackets of credit and debit. Now, during the modeling phase, these could be considered as a seperate set of customers
* For debit values, we see that there is a significant difference in the distribution for churn and non churn and it might be turn out to be an important feature

### Average monthly balance of previous and previous to previous quarters

Now, these 2 variables present deeper historical transactions and would help in understanding the trend during the last 2 quarters

In [ ]:
q_cols = ['average_monthly_balance_prevQ', 'average_monthly_balance_prevQ2']
df1 = pd.DataFrame()

for i in q_cols:
    df1[str('log_')+ i] = np.log(df[i] + 17000)

log_q_cols = df1.columns
df1['churn'] = df['churn']

In [ ]:
sns.pairplot(df1,vars=log_q_cols, hue = 'churn',plot_kws={'alpha':0.5})
plt.show()

The distributions do not have much difference when it comes to churn. 


However, there are some high negative values in the previous to previous quarters due to which there appears to be a lateral shift. However, if you look at the x-axis, it is still at the same scale for both features.

Removing the extreme outliers from the data using the 1 and 99th percentile would help us look at the correct distributions

In [ ]:
# Remove 1st and 99th percentile and plot

df2 = df[['average_monthly_balance_prevQ', 'average_monthly_balance_prevQ2']]

low = .01
high = .99
quant_df = df2.quantile([low, high])
print(quant_df)


In [ ]:
df3 = df2.apply(lambda x: x[(x>quant_df.loc[low,x.name]) & 
                                    (x < quant_df.loc[high,x.name])], axis=0)

In [ ]:
q_cols = ['average_monthly_balance_prevQ', 'average_monthly_balance_prevQ2']
df1 = pd.DataFrame()

for i in q_cols:
    df1[str('log_')+ i] = np.log(df3[i] + 17000)

log_q_cols = df1.columns
df1['churn'] = df['churn']

In [ ]:
sns.pairplot(df1,vars=log_q_cols, hue = 'churn',plot_kws={'alpha':0.5})
plt.show()

Now, we can clearly see that the distributions are very similar for both the variables and and non churning customers have higher average monthly balances in previous 2 quarters

## Demographics and Bank Related Information for customers

Revisiting the description for numerical demographic & Bank related customer variables we have:

#### Numerical:

<b>vintage</b> - Vintage of the customer with the bank in number of days

<b>age</b> - Age of customer

<b>days_since_last_transaction</b> - No of Days Since Last Credit in Last 1 year

#### Ordinal:

<b>dependents</b> - Number of dependents

<b>customer_nw_category</b> - Net worth of customer (3:Low 2:Medium 1:High)

KDE plot can be used for numerical variables on the same axis to quickly compare the distributions for churning and non churning customers. It basically plots the approximate churn rate against each normal variable. This is exactly similar to what we did in the pairplot with distributions but here we would look at them separately since they represent entirely different variables

### Days Since Last Transaction

In [ ]:
# KDE Plot Smoothens out even if there are no values for a value
def kdeplot(feature):
    plt.figure(figsize=(9, 4))
    plt.title("KDE Plot for {}".format(feature))
    ax0 = sns.kdeplot(df[df['churn'] == 0][feature].dropna(), color= 'dodgerblue', label= 'Churn - 0')
    ax1 = sns.kdeplot(df[df['churn'] == 1][feature].dropna(), color= 'orange', label= 'Churn - 1')

kdeplot('days_since_last_transaction')

There is no significant difference between the distributions for churning and non churning customers when it comes to days since last transaction.

### Age & Vintage

In [ ]:
kdeplot('age')

Similarly, age also does not significantly affect the churning rate. However, customers above 80 years of age less likely to churn

In [ ]:
kdeplot('vintage')

For most frequent vintage values, the churning customers are slightly higher, while for higher values of vintage, we have mostly non churning customers which is in sync with the age variable 

<h2>Categorical features</h2>

This dataset has 4 categorical features (gender, occupation, city and branch code) as can be inferred from the data dictionary. Now let us have a look at the the number of unique values for each of them.


In [ ]:
cat_cols = ['gender', 'occupation', 'city','branch_code']

for i in range(0,len(cat_cols)):
    print(str(cat_cols[i]) + " - Number of Unique Values: " + str(df[cat_cols[i]].nunique()))

So, there are a large number of unique values for branch code and city. Gender has 2 unique values while occupation has 7. 

### Univariate Analysis

Let us look at each categorical feature and check distribution

In [ ]:
color = sns.color_palette()

int_level = df['gender'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Gender', fontsize=12)
plt.show()

Amongst the customers, we have more males than females here. Lets check occupation now

In [ ]:
color = sns.color_palette()

int_level = df['occupation'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Occupation', fontsize=12)
plt.show()

Most of the customers are self employed, followed by salaried account holders, retired custumors and very low number of companies.

Now, branch code and city code have a lot of unique values and direct visualization will be difficult. Lets see how:

### City Code & Branch Code

In [ ]:
color = sns.color_palette()

int_level = df['city'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('City', fontsize=12)
plt.show()

Now, let us have a look at the frequencies of the top city codes:

In [ ]:
df['city'].value_counts().head(20)

In [ ]:
# Convert city variable wrt degree of number of customers
df['city_bin'] = df['city'].copy()
counts = df.city.value_counts()
df.city_bin[df['city'].isin(counts[counts > 900].index)] = 3
df.city_bin[df['city'].isin(counts[counts < 900].index) & df['city_bin'].isin(counts[counts >= 350].index)] = 2
df.city_bin[df['city'].isin(counts[counts < 350].index) & df['city_bin'].isin(counts[counts >= 100].index)] = 1
df.city_bin[df['city'].isin(counts[counts < 100].index)] = 0

df['city_bin'] = pd.to_numeric(df['city_bin'], errors='coerce')

In [ ]:
int_level = df['city_bin'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('city bins', fontsize=12)
plt.show()

There are 2 major categories here. Cities with more than 900 occurances and with less than 100 occurances. Similarly we can create bins for branch id and have a look.

In [ ]:
df['branch_code'].value_counts()

In [ ]:
# Convert city variable wrt degree of number of customers
df['branch_bin'] = df['branch_code'].copy()
counts = df.branch_code.value_counts()
df.branch_bin[df['branch_code'].isin(counts[counts >= 100].index)] = 2
df.branch_bin[df['branch_code'].isin(counts[counts < 100].index) & df['branch_bin'].isin(counts[counts >= 50].index)] = 1
df.branch_bin[df['branch_code'].isin(counts[counts < 50].index)] = 0

df['branch_bin'] = pd.to_numeric(df['branch_bin'], errors='coerce')


In [ ]:
df['branch_bin'].value_counts()

In [ ]:
int_level = df['branch_bin'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('branch bins', fontsize=12)
plt.show()

So creating brackets/bins on the basis of frequency is a good idea to quickly analyse a variable with high number of categories

Next, Let us look at some bivariate analysis for categorical variables

### Bivariate Analysis

Lets define a function to quickly compare churn rates for different categories in each feature

In [ ]:
def barplot_percentages(feature):
    #fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 6))
    ax1 = df.groupby(feature)['churn'].value_counts(normalize=True).unstack()
    ax1.plot(kind='bar', stacked='True')
    int_level = df[feature].value_counts()

    plt.figure(figsize=(8,4))
    sns.barplot(int_level.index, int_level.values, alpha=0.8, color=color[1])
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xlabel(str(feature), fontsize=12)
    plt.show()

### Gender

In [ ]:
barplot_percentages("gender")

Does not look like a very significant variable as the ratio of churned customers and others is very similar

### Occupation

In [ ]:
barplot_percentages("occupation")

Self Employed and salaried have higher churn rate and are the major categories.

### Branch Bins

In [ ]:
barplot_percentages("branch_bin")

### City Bins

In [ ]:
barplot_percentages("city_bin")

Here, we see significant difference for different occupations and certainly would be interesting to use as a feature for prediction of churn. However, city and branch codes have little difference amongst the different types of branches

### Dependents

In [ ]:
df['dependents'][df['dependents'] > 3] = 3

In [ ]:
barplot_percentages("dependents")

Most customers have no dependents and hence this variable in itself has low variance so it is of little significance

### Customer Net worth Category

In [ ]:
barplot_percentages("customer_nw_category")

Not much difference in customer net worth category when it comes to churn

### Correlation Heatmap

Lastly, we will look at the correlation heatmap to check what all variables are correlated and to what extent.

In [ ]:
plt.figure(figsize=(12, 6))
df.drop(['customer_id'],
        axis=1, inplace=True)
corr = df.apply(lambda x: pd.factorize(x)[0]).corr()
ax = sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, 
                 linewidths=.2, cmap="YlGnBu")

* The balance features are highly correlated as can be seen from the plot
* Other variables have correlations but on the lower side
* Debit values have the highest correlation amongst the balance features
* Interestingly vintage has a considerable correlation with all the balance features which actually makes sense since older customers will tend to have higher balance

### Conclusions

#### Average customer Profile
Overall a customer at this bank:
* has no dependents
* has been a customer for last 6 years
* predominantly male
* either self employed or salaried customer

#### Conclusion for Churn
* From the sample, around 17% customers are churning
* Current balance and average monthly balance values have a left skewed distribution as observed from the histogram
* No significant difference in distributions for average monthly balance and month end balances
* Bimodal distribution/Double Bell Curve shows that there are 2 different types of customers with 2 brackets of credit and debit. Now, during the modeling phase, these could be considered as a seperate set of customers
* For debit values, we see that there is a significant difference in the distribution for churn and non churn and it might be turn out to be an important feature
* For most frequent vintage values, the churning customers are slightly higher, while for higher values of vintage, we have mostly non churning customers which is in sync with the age variable 
* Gender does not look like a very significant variable as the ratio of churned customers and others is very similar
* Self Employed and salaried have higher churn rate and are the most frequently occuring categories.
* Not much difference in customer net worth category when it comes to churn